# Practical 7 - Part 2A
This project explores the geometry of a single camera. The aim is to take several points on
a plane, and predict where they will appear in the camera image. Based on these observed
points, we will then try to re-estimate the Euclidean transformation relating the plane and
the camera. In practical 2B we will use this code to draw a wireframe cube
on an augmented reality marker.   You should use this
template for your code and fill in the missing sections marked "TO DO"

# Import libraries 

In [4]:
%matplotlib inline
import os 
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio

# TO DO - fill in projectiveCamera and estimatePlanePose functions  (you will have to utilise your solutions for solveAXEqualsZero and calcBestHomography from Part 1)

In [5]:
#The goal of this function is to project points in XCart through projective camera
#defined by intrinsic matrix K and extrinsic matrix T.
def projectiveCamera(K,T,XCart):
    
    # TO DO: Replace this
    # XImCart =

    # TO DO: Convert Cartesian 3d points XCart to homogeneous coordinates XHom
    XHom = np.concatenate((XCart, np.ones((1,XCart.shape[1]))), axis=0)
    
    # TO DO: Apply extrinsic matrix to XHom, to move to frame of reference of camera
    XHom_cam = T@XHom
    # TO DO: Project points into normalized camera coordinates xCamHom (remove 4th row)
    XHom_cam = XHom_cam[0:3,:]
    # TO DO: Move points to image coordinates xImHom by applying intrinsic matrix
    X_image = K@XHom_cam
    # TO DO: Convert points back to Cartesian coordinates xImCart
    X_image = X_image[0:2,:] / np.tile([X_image[2,:]],(2,1))
    return X_image


## Comments
The code above is used to map the points to the frame of reference of camera using extrinsic matrix T and project the points to image coordinates using intrinsic matrix K. 

In [18]:
def solveAXEqualsZero(A):
    # TO DO: Write this routine - it should solve Ah = 0   
    #Compute SVD of matrix A
    u, s, vh = np.linalg.svd(A)
    #set h to the last column of V
    h = vh.T[:,-1]
    return h

# This function should apply the direct linear transform (DLT) algorithm to calculate the best 
# homography that maps the points in pts1Cart to their corresonding matching in pts2Cart
def calcBestHomography(pts1Cart, pts2Cart):    
    # TO DO: replace this
    H = np.zeros([3,3])

    # TO DO: 
    # First convert points into homogeneous representation
    pts1Hom = np.concatenate((pts1Cart[0:2,:], np.ones((1,pts1Cart.shape[1]))), axis=0)
    # Then construct the matrix A, size (n_points,9)  
    #map from 2D -> 2D
    n_points = pts1Cart.shape[1]
    A = np.zeros([2*n_points,9])

    #find the matrix relates to X
    #pts2Cart - dxn
    # value of x in the second coordinate system
    coord2_X = np.tile(pts2Cart[0,:],(3,1)).T
    # value of y in the second coordinate system
    coord2_Y = np.tile(pts2Cart[1,:],(3,1)).T

    
    #concentrate the matrix
    X = np.hstack((pts1Hom.T,np.zeros([n_points,3]),(coord2_X*(-pts1Hom.T))))
    Y = np.hstack((np.zeros([n_points,3]),-pts1Hom.T,(coord2_Y*(pts1Hom.T))))
    
    for i in range(n_points):
        indexRelates_y = 2*i
        indexRelates_x = 2*i + 1
        A[indexRelates_x] = X[i]
        A[indexRelates_y] = Y[i]

    # Solve Ah = 0
    h = solveAXEqualsZero(A)
    # Reshape h into the matrix H, values of h go first into rows of H
    H = np.reshape(h,H.shape)
    return H

In [19]:
# Goal of function is to estimate pose of plane relative to camera (extrinsic matrix)
# given points in image xImCart, points in world XCart and intrinsic matrix K

def estimatePlanePose(XImCart,XCart,K):

    # TO DO: replace this
    #T = 

    # TO DO: Convert Cartesian image points XImCart to homogeneous representation XImHom
    XImHom = np.concatenate((XImCart, np.ones((1,XImCart.shape[1]))), axis=0)
    
    # TO DO: Convert image co-ordinates XImHom to normalized camera coordinates XCamHom    
    XCamHom = np.linalg.inv(K)@XImHom
    
    # TO DO: Estimate homography H mapping homogeneous (x,y) coordinates of positions
    # in real world to XCamHom (convert XCamHom to Cartesian, calculate the homography) -
    # use the routine you wrote for Practical 1B
    XCam = XCamHom[0:2,:] / np.tile([XCamHom[2,:]],(2,1))
    H = calcBestHomography(XCart, XCam)   
    # TO DO: Estimate first two columns of rotation matrix R from the first two
    # columns of H using the SVD       
    
    # Get first two cols which relates to rotation matrix
    H_R = H[:,0:2]
    U, L, Vh = np.linalg.svd(H_R)
    # chose R_1:2 = UVh
    I = np.array([[1,0],[0,1],[0,0]])
    R_12 = U@I@Vh
    
    
    # TO DO: Estimate the third column of the rotation matrix by taking the cross
    # product of the first two columns
    R_3 = np.cross(R_12[:,0],R_12[:,1]).reshape(3,1)
    R = np.hstack((R_12,R_3))
    
        
    # TO DO: Check that the determinant of the rotation matrix is positive - if
    # not then multiply last column by -1.
    det = np.linalg.det(R)
    if det <= 0:
        R[:,2] *= -1  
    
    # TO DO: Estimate the translation t by finding the appropriate scaling factor k
    # and applying it to the third colulmn of H
    k = sum(sum(R[:,0:2]/H[:,0:2]));
    t = k*H[:,-1]
    t = t/6; #average

    # TO DO: Check whether t_z is negative - if it is then multiply t by -1 and
    # the first two columns of R by -1.
    if t[2]<0:
        t=-1*t
        R[:,0:2]=-1*R[:,0:2]
  
    # TO DO: Assemble transformation into matrix form
    T = np.hstack((R,t.reshape(3,1)))
    temp = np.array([[0,0,0,1]])
    T = np.r_[T,temp]
    
    return T 

## Comments
The code above is used to find the estimated rotation matrix and translation matrix from the estimated Homography matrix.

We have

$\lambda \begin{bmatrix}
        x\\ 
        y\\ 
        1
       \end{bmatrix}
       = K
       \begin{bmatrix}
        w_{11} & w_{12} & \tau_{x}\\ 
        w_{21} & w_{22} & \tau_{y}\\ 
        w_{31} & w_{32} & \tau_{z}
       \end{bmatrix}
       \begin{bmatrix}
        u\\ 
        v\\ 
        1
       \end{bmatrix}
       =
       \begin{bmatrix}
       \phi_{11} & \phi_{12} & \phi_{13}\\ 
       \phi_{21} & \phi_{22} & \phi_{23}\\ 
       \phi_{31} & \phi_{32} & \phi_{33}
       \end{bmatrix}
       \begin{bmatrix}
        u\\ 
        v\\ 
        1
       \end{bmatrix}$

where K is the intrinsic matrix of camera.

Factor out the intrinsic parameters
$
\begin{bmatrix}
       \phi'_{11} & \phi'_{12} & \phi'_{13}\\ 
       \phi'_{21} & \phi'_{22} & \phi'_{23}\\ 
       \phi'_{31} & \phi'_{32} & \phi'_{33}
\end{bmatrix}
= \lambda'       
\begin{bmatrix}
w_{11} & w_{12} & \tau_{x}\\ 
w_{21} & w_{22} & \tau_{y}\\ 
w_{31} & w_{32} & \tau_{z}
\end{bmatrix}
$

**Rotation Matrix**

First, to estimate the first two columns of rotation matrix, we compute 

$\begin{bmatrix}
       \phi'_{11} & \phi'_{12} \\ 
       \phi'_{21} & \phi'_{22} \\ 
       \phi'_{31} & \phi'_{32}
\end{bmatrix} = ULV^T$ and choose $R'=UV^T$

Then, we use the cross product of first two columns to find the last column of the rotation matrix.

Furthermore, if the determinant is nagtive, we need to multiply the last column by -1.

**Translation Matrix**

First, we find the translation scaling factor by using

$\lambda' = \frac{\sum_{m=1}^3\sum_{n=1}^2\phi'_{mn}/w_{mn}}{6}$

Then set the translation matrix to be

$\tau = [\phi'_{13},\phi'_{23},\phi'_{33}]^T/ \lambda'$

The translation matrix is also required to be checked weather tz is negative to ensure the valid transformation.


# Once you have completed these functions, use them to estimate the transformation from the plane co-ordinate system to the camera co-ordinate system (i.e. the extrinsic matrix)

In [20]:
# We assume that the intrinsic camera matrix K is known and has values
K = np.array([[640, 0, 320],
             [0, 640, 240],
             [0, 0, 1]])

# We will assume an object co-ordinate system with the Z-axis pointing upwards and the origin
# in the centre of the plane. There are four known points on the plane with coordinates (mm):
XCart = np.array([[-100, -100,  100,  100, 0],
                  [-100,  100,  100, -100, 0],
                  [   0,    0,    0,    0, 0]])

# We assume the correct transformation from the plane co-ordinate system to the
# camera co-ordinate system (extrinsic matrix) is:
T = np.array([[0.9851,  -0.0492,  0.1619,  46.00],
             [-0.1623,  -0.5520,  0.8181,  70.00],
             [0.0490,  -0.8324, -0.5518,  500.89],
             [0,        0,       0,       1]])
  
# TO DO: Use the general pin-hole projective camera model discussed in the lectures to estimate 
# where the four points on the plane will appear in the image.  Fill in the
# details of the function "projectiveCamera" - body of function appears below:

XImCart =  projectiveCamera(K,T,XCart)

# TO DO: Add noise (standard deviation of one pixel in each direction) to the pixel positions
# to simulate having to find these points in a noisy image. Store the results back in xImCart


# TO DO: Now we will take the image points and the known positions on the card and estimate  
# the extrinsic matrix using the algorithm discussed in the lecture.  Fill in the details of 
# the function "estimate plane pose"
TEst = estimatePlanePose(XImCart,XCart,K)


# If you have got this correct, Test should closely resemble T above

In [21]:
TEst

array([[ 9.85520172e-01, -4.93647932e-02,  1.62213157e-01,
         4.60524114e+01],
       [-1.62275327e-01, -5.51968840e-01,  8.17922440e-01,
         7.00797564e+01],
       [ 4.91600358e-02, -8.32402256e-01, -5.51987114e-01,
         5.01460703e+02],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00]])

## Comments
The program first uses extrinsic to map points in defined object co-ordinate system to the reference camera coordinates and then use intrinsic matrix to map points to the image coordinates. 
Then, with the obtained Cartesian coordinates in the image plane and points in the object co-ordinate system, the Homography matrix is calculated. The Homography matrix is used to recover the transformation matrix. As we can see from the result, the estimated transformation matrix closely resembles the defined transformation matrix.